In [2]:
''' Import packages. '''
# Time packages
import cftime, datetime, time
# Numerical analysis packages
import numpy as np, random, scipy
# Local data storage packages
import dill, os, pickle
# Data structure packages
import pandas as pd, xarray as xr

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [16]:
def tc_storage(model, experiment, year_range, storm_type, benchmarking=True):
    
    ''' 
    Function to build unified xArray Dataset to hold model and track data for tracked TCs in a given model and experiment.
    
    Input(s):
    - model (str):                name of model to access (typically "AM2.5" or "HIRAM")
    - experiment (str):           name of experiment to access (typically "control", "ktc", "plus2K", etc.)
    - year_range (tuple of ints): 2-element tuple with a start and end year
    - storm_type (str):           type of storm to evaluate from TC tracks data ("TS" for all storms or "C15w" for hurricanes)
    - benchmarking (bool):        boolean to enable time benchmarking
    Output(s):
    - dirs (tuple):               2-element tuple with pathnames for the model- and experiment-specific GCM and track directories.
    '''
    
    # Define paths to model- and experiment-specific data
    model_dir, track_dir = dirnames(model, experiment)
    # Define date range
    date_range = [cftime.DatetimeNoLeap(year=year, month=1, day=1, hour=0) for year in range(min(year_range), max(year_range))]
    # Iterate over years. This is performed because data is saved into year-specific files.
    for year in date_range:
        

In [4]:
def dirnames(model, experiment):

    ''' 
    Function to store pathnames for selected models and experiments. 
    
    Input(s):
    - model (str):  name of model to access (typically "AM2.5" or "HIRAM").
    Output(s):
    - dirs (tuple): 2-element tuple with pathnames for the model- and experiment-specific GCM and track directories.
    '''
    
    # Define experiment-specific pathnames
    experiments = {'control': {'AM2.5': 'CTL1990s_tigercpu_intelmpi_18_540PE',
                               'HIRAM': 'CTL1990s_v201910_tigercpu_intelmpi_18_540PE'},
                   'ktc': {'AM2.5': 'CTL1990s_killtc13-13-15_tigercpu_intelmpi_18_540PE',
                           'HIRAM': 'CTL1990s_v201910_killtc13-13-15_tigercpu_intelmpi_18_540PE'}}
    # Model directories
    model_dirs = {'AM2.5': {'control': '/tigress/wenchang/analysis/TC/AM2.5/CTL1990s_tigercpu_intelmpi_18_540PE/model_out/POSTP',
                            'ktc': '/tigress/wenchang/analysis/TC/AM2.5ktc2/CTL1990s_killtc13-13-15_tigercpu_intelmpi_18_540PE/modelout/POSTP'},
                  'HIRAM': {'control': '/tigress/wenchang/analysis/TC/HIRAM/CTL1990s_v201910_tigercpu_intelmpi_18_540PE/model_out/POSTP',
                            'ktc': '/tigress/wenchang/analysis/TC/HIRAMktc2/CTL1990s_v201910_killtc13-13-15_tigercpu_intelmpi_18_540PE/modelout/POSTP'}}
    # Track directories
    # Potential issue: ask Wenchang about using track .nc files in the same parent directory tree as the model data
    track_dirs = {'AM2.5': {'control': '/tigress/wenchang/MODEL_OUT/AM2.5/CTL1990s_tigercpu_intelmpi_18_540PE/analysis_lmh/cyclones_gav_ro110_1C_330k',
                            'ktc': '/tigress/wenchang/MODEL_OUT/AM2.5ktc2/CTL1990s_killtc13-13-15_tigercpu_intelmpi_18_540PE/analysis_lmh/cyclones_gav_ro110_1C_330k'},
                  'HIRAM': {'control': '/tigress/wenchang/MODEL_OUT/HIRAM/CTL1990s_v201910_tigercpu_intelmpi_18_540PE/analysis_lmh/cyclones_gav_ro110_2p5C_330k',
                            'ktc': '/tigress/wenchang/MODEL_OUT/HIRAMktc2/CTL1990s_v201910_killtc13-13-15_tigercpu_intelmpi_18_540PE/analysis_lmh/cyclones_gav_ro110_1C_330k'}}

    dirs = (model_dirs[model][experiment], track_dirs[model][experiment])
    
    return dirs

In [106]:
def retrieve_tracked_TCs(dirname, storm_type, year_range):

    '''
    Function to collect tracked TC data and add derived data, such as duration and storm speed.
    
    Input(s):
    - dirname (str):              name of directory containing files of interest
    - storm_type (str):           type of storm to evaluate from TC tracks data ("TS" for all storms or "C15w" for hurricanes)
    - year_range (tuple of ints): 2-element tuple with a start and end year
    Output(s):
    - data (Pandas DataFrame):    Pandas DataFrame with tracked TC data
    '''
    
    ''' File collection. '''
    # Get filenames for all files within the specified directory 
    # Filenames will correspond to the determined storm type
    fnames = [[os.path.join(dirname, file, 'Harris.TC', f) for f in os.listdir(os.path.join(dirname, file, 'Harris.TC')) 
               if '{0}.world'.format(storm_type) in f]
               for file in sorted(os.listdir(dirname))]
    # Compress 2D list to 1D list
    fnames = [item for sublist in fnames for item in sublist]

    # Select files with dates within 'year_range'
    # Note: the '+ 1900' is added because tracked TCs are on the 2000 year range, whereas model output is on the 100 year range
    fnames = [f for f in fnames 
              if min(year_range) + 1900 <= pd.to_datetime(f.split('.')[-2].split('-')[0]).year < max(year_range) + 1900]
    
    # Concatenate all tracked TC data from the filename list
    data = pd.concat([lmh_parser(os.path.join(dirname, fname)) for fname in fnames])

    ''' Derived data algorithm. '''
    # Initialize empty duration column to populate iteratively
    data[['duration', 'velocity']] = np.nan
    # Iterate through each unique storm (identify by 'storm_id') and get duration
    for storm in data['storm_id'].unique():
        # Get difference between minimum and maximum timestamps
        dt = (data.loc[data['storm_id'] == storm]['time'].max() - data.loc[data['storm_id'] == storm]['time'].min())
        # Convert difference timedelta into hours
        dt = dt.days + dt.seconds/86400
        # Add duration to the DataFrame for the corresponding storm
        data.loc[data['storm_id'] == storm, 'duration'] = dt
        
    ''' Storm speed algorithm. '''
    
        
    # Rename columns for future addition into xArray Dataset
    data = data.rename(columns={'lon': 'center_lon', 'lat': 'center_lat', 'flag': 'core_temp', 'slp': 'min_slp'})
        
    return data

In [29]:
def lmh_parser(path):
    
    ''' 
    This method parses through text files from Lucas Harris' run outputs (held in directories titled 'analysis_lmh') 
    and produces an output DataFrame. 
    
    Input(s):
    - path (str):            path containing raw tracker data from Lucas Harris' runs.
    Output(s):
    - df (Pandas DataFrame): Pandas DataFrame containing tracked TC data
    '''
    
    # Create file object instance
    fobj = open(path, 'r').readlines()
    # Initialize dictionary to hold data
    data = {'storm_num': {}}
    # Initialize storm counter
    count = 1
    # Iterate through text file
    for line in fobj:
        # Extract information from the line
        content = line.strip()
        # Creates new storm-specific dict in the parent dict. The '+++' demarcates a new storm.
        if '+++' in line:
            storm_num = '{0:04d}'.format(count)
            data['storm_num'][storm_num] = {'storm_id': [], 'time': [], 'lon': [], 'lat': [], 'slp': [], 'max_wnd': [], 'flag': []}
            count += 1
        # Populates the storm-specific dict
        else:
            storm_num = '{0:04d}'.format(count-1) 
            tc_info = [x for x in content.split(' ') if x]
            year = tc_info[0][0:4] # get 4-digit year
            data['storm_num'][storm_num]['storm_id'].append('{0}-{1:04d}'.format(year, count-1))
            data['storm_num'][storm_num]['time'].append(tc_info[0])
            data['storm_num'][storm_num]['lon'].append(tc_info[1])
            data['storm_num'][storm_num]['lat'].append(tc_info[2])
            data['storm_num'][storm_num]['slp'].append(tc_info[3])
            data['storm_num'][storm_num]['max_wnd'].append(tc_info[4])
            data['storm_num'][storm_num]['flag'].append(tc_info[5])
    
    try:
        # Converts the dictionary into a DataFrame
        df = pd.concat({k: pd.DataFrame(v).T for k, v in data.items()}, axis=1)['storm_num']
        df = df.explode(df.columns.to_list()).reset_index().rename(columns={'index': 'storm_num'})
        # Re-cast column data types
        df = df.astype({'lon': 'float', 'lat': 'float', 'slp': 'float', 'max_wnd': 'float', 'flag': 'float'})
    except:
        df = pd.DataFrame(columns=['storm_id', 'time', 'lon', 'lat', 'slp', 'max_wnd', 'flag'])
    
    ''' DataFrame refinement. '''
    # Remove cold-core data points (flag == -1)
    df = df.loc[df['flag'] != -1]
    # Convert timestamps to datetime objects
    df['time'] = pd.to_datetime(df['time'], format='%Y%m%d%H')
    
    return df

In [94]:
def coords_to_dist(a, b):
    ''' Convert coordinates to distance in meters. '''
    
    R = 6371e3
    
    lon_a, lat_a = np.array(a)*np.pi/180
    lon_b, lat_b = np.array(b)*np.pi/180
    
    dlon, dlat = lon_b - lon_a, lat_b - lat_a
    
    a = np.sin(dlat/2)**2 + np.cos(lat_a)*np.cos(lat_b)*np.sin(dlon/2)**2    
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    distance = R*c
    
    return distance

In [107]:
dirname = '/tigress/wenchang/MODEL_OUT/HIRAMktc2/CTL1990s_v201910_killtc13-13-15_tigercpu_intelmpi_18_540PE/analysis_lmh/cyclones_gav_ro110_1C_330k'
df = retrieve_tracked_TCs(dirname, 'C15w', year_range=(101, 105))
print(df)
del dirname, df

    storm_num   storm_id                time  center_lon  center_lat  min_slp  \
0        0001  2001-0001 2001-05-19 12:00:00      292.70       20.61  1005.16   
2        0001  2001-0001 2001-05-20 00:00:00      292.25       21.64   998.55   
3        0001  2001-0001 2001-05-20 06:00:00      291.91       22.53   995.11   
4        0001  2001-0001 2001-05-20 12:00:00      291.71       23.09   996.21   
5        0001  2001-0001 2001-05-20 18:00:00      291.40       23.78   994.34   
..        ...        ...                 ...         ...         ...      ...   
171      0003  2004-0003 2004-10-01 18:00:00      115.25       26.16  1006.58   
172      0003  2004-0003 2004-10-02 00:00:00      115.09       27.03  1007.37   
173      0003  2004-0003 2004-10-02 06:00:00      115.38       27.59  1005.68   
174      0003  2004-0003 2004-10-02 12:00:00      115.43       27.96  1007.71   
175      0003  2004-0003 2004-10-02 18:00:00      116.20       28.73  1008.63   

     max_wnd  core_temp  du